# CLEAN Algorithm

Based on "The CLEAN type algorithms for radar signal processing" by [Kulpa (2008)](https://ieeexplore.ieee.org/document/4669567).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pluto_sdr_pr.ioutils
import pluto_sdr_pr.processing

In [ ]:
c = 3e8

In [ ]:
def calc_plot_parameters(sampling_rate, cpi, center_freq, max_distance=50e3, max_speed=280):
    num_samples = int(cpi * sampling_rate)
    max_delay = int(max_distance / c * sampling_rate)
    max_doppler = int(max_speed * cpi * center_freq / c)
    return (num_samples, max_delay, max_distance, max_doppler, max_speed)

In [ ]:
file_path = "data/pluto_b_surv.2021-08-03T17_49_05_196.sdriq"

with open(file_path, 'rb') as fid:
    _, header = pluto_sdr_pr.ioutils.read_sdriq_samples(fid, 0)
    fid.seek(0)

    cpi = 1 # in seconds
    num_samples, max_delay, _, max_doppler, max_speed = calc_plot_parameters(header["sample_rate"], cpi, header["center_frequency"])

    n, _ = pluto_sdr_pr.ioutils.read_sdriq_samples(fid, num_samples)

    amb = pluto_sdr_pr.processing.fast_ambiguity(max_delay, max_doppler, n, n)

In [ ]:
curr_amb = amb
curr_n = n

ambs = np.empty_like(amb, shape=(5,) + amb.shape)

for idx in range(ambs.shape[0]):
    ambs[idx, :, :] = curr_amb

    offset = 0
    clutter_delay, clutter_doppler = np.unravel_index(np.argmax(curr_amb[offset:, :], axis=None), curr_amb.shape)
    clutter_delay += offset
    clutter_doppler -= max_doppler // 2
    clutter_range = clutter_delay * c / header["sample_rate"]
    clutter_speed = clutter_doppler * c / (cpi * header["center_frequency"])

    clutter_model = np.roll(np.pad(curr_n, pad_width=(0, clutter_delay)), clutter_delay)[0:curr_n.shape[0]] * np.exp(-2j * np.pi * clutter_doppler * clutter_speed)

    clutter_amplitude = np.dot(n, clutter_model) / np.dot(clutter_model, clutter_model)
    print(f"Clutter peak at ({clutter_range} m, {clutter_speed} m/s, Energy: {clutter_amplitude:.3f})")

    curr_n = curr_n - clutter_model * clutter_amplitude
    curr_amb = pluto_sdr_pr.processing.fast_ambiguity(max_delay, max_doppler, curr_n, curr_n)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(20, 10))
for ax in axs:
    ax.set_title("Range/Doppler Map")
    ax.set_xlabel("bistatic range [km]")
    ax.set_ylabel("bistatic velocity [m/s]")
    xticks = np.linspace(0, 50e3, 6, endpoint=True)
    ax.set_xticks(xticks / c * header["sample_rate"])
    ax.set_xticklabels(map(lambda x: f"{x / 1e3:.0f} km", xticks))

    yticks = np.linspace(-max_speed, max_speed, 15, endpoint=True)
    ax.set_yticks((yticks + max_speed) * cpi * header["center_frequency"] / c)
    ax.set_yticklabels(map(lambda y: f"{y:.0f} m/s", yticks))

axs[0].imshow(10 * np.log10(np.abs(amb.T)), interpolation='hanning', aspect='auto');
axs[1].imshow(10 * np.log10(np.abs(ambs[-1].T)), interpolation='hanning', aspect='auto');